# Notebook for GWAS analysis

Rpy2 allows to exectute R commands inside a Python notebook

In [1]:
%load_ext rpy2.ipython

/home/pierre/.soft/miniconda3/envs/TP-GWAS/lib/python3.7/site-packages/rpy2/ipython/rmagic.py:76: UserWarning: The Python package 'pandas' is strongly recommended when using `rpy2.ipython`. Unfortunately it could not be loaded, as we did not manage to load 'numpy' first.
  warnings.warn(' '.join(("The Python package 'pandas' is strongly",


# Merge cases and controls files

In [2]:
%%bash

plink --bfile input/ic_cases --make-bed --out output/ic_cases
plink --bfile input/ic_controls --make-bed --out output/ic_controls
plink --bfile output/ic_cases --exclude input/toexclude.txt --make-bed --out output/ic_cases_autosomes
plink --bfile output/ic_controls --bmerge output/ic_cases_autosomes.bed output/ic_cases_autosomes.bim output/ic_cases_autosomes.fam --make-bed --out output/t1dcc
plink --bfile output/ic_controls --exclude input/rs3754055.txt --make-bed --out input/ic_controls2
plink --bfile output/ic_cases_autosomes --exclude input/rs3754055.txt --make-bed --out input/ic_cases_autosomes2
plink --bfile output/ic_controls --flip output/t1dcc-merge.missnp --make-bed --out output/ic_controls_flipped
plink --bfile output/ic_controls_flipped --bmerge input/ic_cases_autosomes2.bed input/ic_cases_autosomes2.bim input/ic_cases_autosomes2.fam --make-bed --out output/t1dcc

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/ic_cases.log.
Options in effect:
  --bfile input/ic_cases
  --make-bed
  --out output/ic_cases

31930 MB RAM detected; reserving 15965 MB for main workspace.
196524 variants loaded from .bim file.
241 people (127 males, 114 females) loaded from .fam.
241 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 3 founders and 238 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.964361.
196524 variants and 241 people pass filters and QC.
Among remaining phenotypes, 241 are cases and 0 are controls.
--make-bed to output/ic_cases.bed + outp

commands treat these as missing.
treat these as missing.
Error: 5 variants with 3+ alleles present.
* If you believe this is due to strand inconsistency, try --flip with
  output/t1dcc-merge.missnp.
  (Warning: if this seems to work, strand errors involving SNPs with A/T or C/G
  alleles probably remain in your data.  If LD between nearby SNPs is high,
  --flip-scan should detect them.)
* If you are dealing with genuine multiallelic variants, we recommend exporting
  that subset of the data to VCF (via e.g. '--recode vcf'), merging with
  another tool/script, and then importing the result; PLINK is not yet suited
  to handling them.
1434 more same-position warnings: see log file.


# Clean SNPs

In [3]:
%%bash

plink --bfile output/t1dcc --geno 0.05 --make-bed --out output/filter1
plink --bfile output/filter1 --test-missing --pfilter 0.00001 --out output/missingcc
plink --bfile output/filter1 --exclude output/missingcc.missing --make-bed --out output/filter2
plink --bfile output/filter2 --hwe 0.0001 --make-bed --out output/filter3
plink --bfile output/filter3 --maf 0.01 --make-bed --out output/filter4

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/filter1.log.
Options in effect:
  --bfile output/t1dcc
  --geno 0.05
  --make-bed
  --out output/filter1

31930 MB RAM detected; reserving 15965 MB for main workspace.
192673 variants loaded from .bim file.
598 people (309 males, 289 females) loaded from .fam.
598 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 360 founders and 238 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.748747.
78405 variants removed due to missing genotype data (--geno).
114268 variants and 598 people pass filters and QC.
Among remaining phenotypes, 

# Clean subjects

In [4]:
%%bash

plink --bfile output/filter4 --mind 0.05 --make-bed --out output/filter5

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/filter5.log.
Options in effect:
  --bfile output/filter4
  --make-bed
  --mind 0.05
  --out output/filter5

31930 MB RAM detected; reserving 15965 MB for main workspace.
112085 variants loaded from .bim file.
598 people (309 males, 289 females) loaded from .fam.
598 phenotype values loaded from .fam.
10 people removed due to missing genotype data (--mind).
IDs written to output/filter5.irem .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 360 founders and 228 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.998742.
112085 variants and 588 p

# Look for relatedness with KING

In [5]:
%%bash

king -b output/filter5.bed --kinship --related --prefix output/filter5
king -b output/filter5.bed --kinship --unrelated --prefix output/filter5

plink --bfile output/filter5 --keep output/filter5unrelated.txt --make-bed --out output/filter6

KING 2.2.3 - (c) 2010-2019 Wei-Min Chen

The following parameters are in effect:
                   Binary File : output/filter5.bed (-bname)

Additional Options
         Close Relative Inference : --related [ON], --duplicate
   Pairwise Relatedness Inference : --kinship [ON], --ibdseg, --ibs, --homog
              Inference Parameter : --degree
         Relationship Application : --unrelated, --cluster, --build
                        QC Report : --bysample, --bySNP, --roh, --autoQC
                     QC Parameter : --callrateN, --callrateM
             Population Structure : --pca, --mds
              Structure Parameter : --projection
              Disease Association : --tdt
   Quantitative Trait Association : --mtscore
                Association Model : --trait [], --covariate []
            Association Parameter : --invnorm, --maxP
               Genetic Risk Score : --risk, --model [], --prevalence, --noflip
              Computing Parameter : --cpus
                   Option

# First association test

In [6]:
%%bash

plink --bfile output/filter6 --assoc --adjust --out output/allelictest

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/allelictest.log.
Options in effect:
  --adjust
  --assoc
  --bfile output/filter6
  --out output/allelictest

31930 MB RAM detected; reserving 15965 MB for main workspace.
112085 variants loaded from .bim file.
572 people (294 males, 278 females) loaded from .fam.
572 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 358 founders and 214 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.998751.
112085 variants and 572 people pass filters and QC.
Among remaining phenotypes, 215 are cases and 357 are controls.
Writing C/C --assoc re

# Start R

In [7]:
%%R
library(qqman)
sessionInfo()

/home/pierre/.soft/miniconda3/envs/TP-GWAS/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/home/pierre/.soft/miniconda3/envs/TP-GWAS/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: For example usage please run: vignette('qqman')

  warnings.warn(x, RRuntimeWarning)
/home/pierre/.soft/miniconda3/envs/TP-GWAS/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Citation appreciated but not required:

  warnings.warn(x, RRuntimeWarning)
/home/pierre/.soft/miniconda3/envs/TP-GWAS/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Turner, S.D. qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. biorXiv DOI: 10.1101/005165 (2014).

  warnings.warn(x, RRuntimeWarning)


R version 3.6.0 (2019-04-26)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS:   /home/pierre/.soft/miniconda3/envs/TP-GWAS/lib/R/lib/libRblas.so
LAPACK: /home/pierre/.soft/miniconda3/envs/TP-GWAS/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=fr_FR.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=fr_FR.UTF-8        LC_COLLATE=fr_FR.UTF-8    
 [5] LC_MONETARY=fr_FR.UTF-8    LC_MESSAGES=fr_FR.UTF-8   
 [7] LC_PAPER=fr_FR.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=fr_FR.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] qqman_0.1.4

loaded via a namespace (and not attached):
[1] MASS_7.3-51.6   compiler_3.6.0  calibrate_1.7.7


In [8]:
%%R
gwasoutputults <- read.table("output/allelictest.assoc",header=T)
head(gwasResults)

  SNP CHR BP         P
1 rs1   1  1 0.9148060
2 rs2   1  2 0.9370754
3 rs3   1  3 0.2861395
4 rs4   1  4 0.8304476
5 rs5   1  5 0.6417455
6 rs6   1  6 0.5190959


In [9]:
%%R 
png("output/manhattan.png")
manhattan(gwasResults)
dev.off()

png 
  2 


![](output/manhattan.png)

In [10]:
%%R
png("output/qqplot.png")
qq(gwasResults$P, main="Q-Qplot of P-values")
dev.off()

png 
  2 


![](output/qqplot.png)

# Perform MDS analysis

In [11]:
%%bash

plink --bfile output/filter6 --exclude input/mhc_range.txt --range --make-bed --out output/withoutmhc
plink --bfile output/withoutmhc --indep 50 5 2 --out output/indep
plink --bfile output/withoutmhc --extract output/indep.prune.in --make-bed --out output/withoutld
king -b output/withoutld.bed --mds --ibs --prefix output/stratifwithoutld

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/withoutmhc.log.
Options in effect:
  --bfile output/filter6
  --exclude input/mhc_range.txt
  --make-bed
  --out output/withoutmhc
  --range

Note: --range flag deprecated.  Use e.g. "--extract range <filename>".
31930 MB RAM detected; reserving 15965 MB for main workspace.
112085 variants loaded from .bim file.
572 people (294 males, 278 females) loaded from .fam.
572 phenotype values loaded from .fam.
--exclude range: 7084 variants excluded.
--exclude range: 105001 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 358 founders and 214 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949

# Start R

In [12]:
%%R
mds <- read.table("output/stratifwithoutldpc.txt", header=F)
png("output/mds.png")
par(mfrow=c(2,2))
plot(mds$V7, mds$V8, type="n", main="MDS vectors 1 and 2 in Controls")
points(mds$V7[1:357], mds$V8[1:357], col="black")
plot(mds$V7, mds$V8, type="n",main="MDS vectors 1 and 2 in Cases" )
points(mds$V7[358:577], mds$V8[358:577], col="red")
plot(mds$V8, mds$V9, type="n", main="MDS vectors 2 and 3 in Controls")
points(mds$V8[1:357], mds$V9[1:357], col="black")
plot(mds$V8, mds$V9, type="n", main="MDS vectors 2 and 3 in Cases")
points(mds$V8[358:577], mds$V9[358:577], col="red")
dev.off()

png 
  2 


![](output/mds.png)

# 2nd association test, after MDS

In [13]:
%%bash

plink --bfile output/filter6 --logistic hide-covar --adjust --covar output/stratifwithoutldpc.txt --covar-number 7-11 --ci 0.95 --out output/logistic_mds_PC5

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/logistic_mds_PC5.log.
Options in effect:
  --adjust
  --bfile output/filter6
  --ci 0.95
  --covar output/stratifwithoutldpc.txt
  --covar-number 7-11
  --logistic hide-covar
  --out output/logistic_mds_PC5

31930 MB RAM detected; reserving 15965 MB for main workspace.
112085 variants loaded from .bim file.
572 people (294 males, 278 females) loaded from .fam.
572 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
--covar: 5 out of 24 covariates loaded.
Before main variant filters, 358 founders and 214 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.998751.


# Start R

In [14]:
%%R
gwasResultsAfterMDS <- read.table("output/logistic_mds_PC5.assoc.logistic", header=T)
head(gwasResultsAfterMDS)
library(qqman)

png("output/manhattan_mds.png")
manhattan(gwasResultsAfterMDS[! is.na(gwasResultsAfterMDS$P),])
dev.off()

png 
  2 


![](output/manhattan_mds.png)

In [15]:
%%R
png("output/qqplot_mds.png")
qq(gwasResultsAfterMDS$P, main="Q-Qplot of P-values")
dev.off()

png 
  2 


![](output/qqplot_mds.png)

# Trios analysis

In [16]:
%%bash

plink --bfile input/ic_trio --tdt --adjust --out output/tdttrio

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/tdttrio.log.
Options in effect:
  --adjust
  --bfile input/ic_trio
  --out output/tdttrio
  --tdt

31930 MB RAM detected; reserving 15965 MB for main workspace.
140197 variants loaded from .bim file.
685 people (359 males, 326 females) loaded from .fam.
685 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 284 founders and 401 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.913213.
140197 variants and 685 people pass filters and QC.
Among remaining phenotypes, 241 are cases and 444 are controls.
--tdt: Report written to output/t

# Start R

In [17]:
%%R
library(qqman)
tdt <- read.table("output/tdttrio.tdt", header=T)
head(tdt)

  CHR        SNP      BP A1 A2  T  U     OR  CHISQ      P A.U_PAR CHISQ_PAR
1   1 rs61733845 1118275  A  G 11 16 0,6875 0,9259 0,3359     0:0        NA
2   1  rs1320571 1120431  A  G 10 12 0,8333 0,1818 0,6698     0:2         2
3   1  rs9729550 1135242  G  T 36 37 0,9730 0,0137 0,9068     0:2         1
4   1  rs1815606 1140435  A  C 35 42 0,8333 0,6364 0,4250     0:2         1
5   1  rs7515488 1163804  T  C 21 31 0,6774 1,9230 0,1655     0:0        NA
6   1 rs11260562 1165310  T  C 16 21 0,7619 0,6757 0,4111     0:1         1
   P_PAR CHISQ_COM  P_COM
1     NA    0,9259 0,3359
2 0,1573    0,6667 0,4142
3 0,3173    0,1169 0,7324
4 0,3173    1,0000 0,3173
5     NA    1,9230 0,1655
6 0,3173    0,9474 0,3304


In [18]:
%%R
png("output/manhattan_trios.png")
manhattan(tdt[! is.na(tdt$P),], p="P")
dev.off()

png 
  2 


![](output/manhattan_trios.png)

In [19]:
%%R
png("output/qqplot_trios.png")
qq(tdt$P)
dev.off()

png 
  2 


![](output/qqplot_trios.png)